<a href="https://colab.research.google.com/github/ThatiFlowers/Treinamento/blob/main/C%C3%B3pia_de_CRUD_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# Importando sql e criando conexões e cursores

import sqlite3

def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao

def create_tables():
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS produtos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        preco REAL NOT NULL,
        categoria TEXT NOT NULL
    )
    """)

    conexao.commit()
    conexao.close()

create_tables()


In [34]:
from os import name
# Criando a classe de produtos/ repositório(domínio)
class Produto:
    def __init__(self,nome,preco,categoria):
        self.nome = nome
        self.preco = preco
        self.categoria = categoria
    def __repr__(self):
        return f"Produto(id={self.id}, nome='{self.nome}', preco={self.preco}, categoria='{self.categoria}')"

In [35]:
# Criando uma pasta no ambiente notebook
!mkdir -p repository

In [36]:
def cadastrar_produto(produto):
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute(
        "INSERT INTO produtos (nome, preco, categoria) VALUES (?, ?, ?)",
        (produto.nome, produto.preco, produto.categoria)
    )

    conexao.commit()
    conexao.close()


In [37]:
def limpar_produtos():
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute("DELETE FROM produtos")
    conexao.commit()
    conexao.close()



In [38]:
#teste

produto1 = Produto("cocavinho",12, "bebida")
produto2 = Produto("bolorosa",12.59,"comida")
produto3 = Produto("brownie", 11, "comida")
cadastrar_produto(produto1)
cadastrar_produto(produto2)
cadastrar_produto(produto3)

In [39]:
def listar_produtos():
  conexao = conexao_bd()
  cursor = conexao.cursor()
  cursor.execute("select * from produtos")
  rows = cursor.fetchall()
  conexao.close()
  return rows

print("Produtos cadastrados:")
for produto_data in listar_produtos():
    # Assuming the order is id, nome, preco, categoria based on your create_tables()
    print(f"ID: {produto_data[0]}, Nome: {produto_data[1]}, Preço: {produto_data[2]}, Categoria: {produto_data[3]}")

Produtos cadastrados:
ID: 1, Nome: cocavinho, Preço: 12.0, Categoria: bebida
ID: 2, Nome: bolorosa, Preço: 12.59, Categoria: comida
ID: 3, Nome: brownie, Preço: 11.0, Categoria: comida
ID: 4, Nome: cocavinho, Preço: 12.0, Categoria: bebida
ID: 5, Nome: bolorosa, Preço: 12.59, Categoria: comida
ID: 6, Nome: brownie, Preço: 11.0, Categoria: comida


In [40]:
for p in listar_produtos():
    print(p[1], p[2], p[3])

cocavinho 12.0 bebida
bolorosa 12.59 comida
brownie 11.0 comida
cocavinho 12.0 bebida
bolorosa 12.59 comida
brownie 11.0 comida


In [41]:
# covertendo em uma lista de produtos
def converter_rows_para_produtos(rows):
    lista_de_produtos = []
    for row in rows:
        # formato: (id, nome, preco, categoria)
        p = Produto(row[1], row[2], row[3])
        lista_de_produtos.append(p)
    return lista_de_produtos

def produtos_id(id):
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute("select * from produtos where id =?", (id,))
    row = cursor.fetchone() # Changed fetchall to fetchone as we expect a single product by ID
    conexao.close()
    if row:
        p = Produto(row[1], row[2], row[3])
        return p
    return None # Retorna "none" se o produto for encontrado com ID conhecido

In [42]:
# Crie uma pasta 'models'no ambiente notebook
!mkdir -p models

In [43]:
pip install Flask

In [44]:
from flask import Flask, request, jsonify

# Initialize Flask application
app = Flask(__name__)

# Define GET and POST endpoint for /produtos
@app.route('/produtos', methods=['GET', 'POST'])
def handle_produtos():
    if request.method == 'GET':
        produtos_db = listar_produtos()
        produtos_json = []
        for p_data in produtos_db:
            produtos_json.append({
                'id': p_data[0],
                'nome': p_data[1],
                'preco': p_data[2],
                'categoria': p_data[3]
            })
        return jsonify(produtos_json)

    elif request.method == 'POST':
        data = request.get_json()
        if not data or not all(k in data for k in ('nome', 'preco', 'categoria')):
            return jsonify({'error': 'Dados incompletos para o produto'}), 400

        nome = data['nome']
        preco = data['preco']
        categoria = data['categoria']

        try:
            novo_produto = Produto(nome, preco, categoria)
            cadastrar_produto(novo_produto)
            return jsonify({'message': 'Produto cadastrado com sucesso', 'produto': {'nome': nome, 'preco': preco, 'categoria': categoria}}), 201
        except Exception as e:
            return jsonify({'error': str(e)}), 500

In [45]:
from flask import Flask, request, jsonify
from threading import Thread

app = Flask(__name__)

@app.route('/produtos', methods=['GET'])
def listar():
    return jsonify([{"id": 1, "nome": "Café", "preco": 5.0}])

def run():
    app.run(port=5000)

thread = Thread(target=run)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [46]:
pip install pyngrok


In [47]:
from pyngrok import ngrok

# Substitua 'YOUR_NGROK_AUTHTOKEN' pelo seu token real do ngrok.
# Você pode obter seu token em https://dashboard.ngrok.com/get-started/38BdfOUu8M7binCVvv96swqp16E_39xoGZGyn2e19rYYEYtg7
ngrok.set_auth_token("38BdfOUu8M7binCVvv96swqp16E_39xoGZGyn2e19rYYEYtg7")

public_url = ngrok.connect(5000)
print(public_url)

NgrokTunnel: "https://decinormal-unhypocritically-rubin.ngrok-free.dev" -> "http://localhost:5000"


In [48]:
public_url = ngrok.connect(5000)
print(public_url)



NgrokTunnel: "https://decinormal-unhypocritically-rubin.ngrok-free.dev" -> "http://localhost:5000"
